# 0 摘要

推荐系统或广告中的CVR预估是非常重要的一个环节。尽管目前CVR预估使用各种深度学习模型达到了SOTA视屏，但仍然在实际中面临着task-specific问题。例如，传统CVR预估模型中训练数据是被曝光的点击样本（点击是转化的先决条件），但在预测时却要对整个样本被曝光样本进行预测，这会造成sample selection bias问题；另外，CVR预估中样本的极端稀疏也使得预测非常困难。

本文通过利用用户action的序列信息：从impression到click到conversion，提出了ESMM(Entire Space Multi-task Model）模型，该模型可以：
- 消除样本选择偏差与数据稀疏性问题
- 采用feature representation transfer learning策略

# 1 简介

CVR预估在ranking系统中非常重要，例如在OCPC广告中预估CVR来调整bid price从而达到平台和广告主的双赢；在推荐系统中CVR也是平衡用户点击偏好与购买偏好的一个重要指标。

本文以电子商务网站的推荐系统CVR预估模型为例，给定推荐items，用户会点击感兴趣的item并在后续产生购买行为。一个用户的actions可以被表示为：impression -> click  -> conversion. 进而CVR预估可以被建模为：

$$pCVR = p(conversion|click, impression)$$

但实际的CVR预估中会面临很多问题：
- SSB：sample selection bias。即训练数据采用曝光后的点击样本，但待预测的样本是整个被曝光的物品空间

![](pictures/esmm-ssb.png)

- DS：Data Sparsity问题。CVR模型的训练数据量是远远小于CTR任务的，所以样本稀疏性对于CVR模型的训练是非常困难的。

目前学界也提出了一些解决办法：
- Hierarchical estimators on difference features are built and combined with LR to solve DS problem，但是需要人工先验来构建hierarchical structures，对于超大规模的推荐系统是不现实的
- 对rare class examples进行过采样可以缓解数据稀疏性，但是对于采样比例非常敏感
- AMAN(All Missing As Negative)方法对曝光未点击的样本进行随机采样作为负例，它可以在一定程度上消除SSB问题，但会使得CVR结果使用被低估
- Unbiased method通过对观察样本进行rejection sampling来拟合真实的分布，这种方法当对样本进行加权时会遇到数值不稳定的情况

本文提出的方法，Entire Space Muti-task Model：
- 引入两个辅助任务，即CTR和CTCVR的预估
- ESMM采用预估pCTR和pCTCVR的方式来得到pCVR，$pCVR*pCTR=pCTCVR$
- CVR和CTR的网络对于feature representation的参数是共享的，CTR训练的样本集更大，因此能进一步给CVR预估带来更多信息，通过这种transfer learning来缓解DS问题

# 2 方法

## 2.1 Notation

- 定义样本空间：$\mathcal{S}=\{(x_i, y_i)\rightarrow z_i\}|_{i=1}^N$
- 特征向量：$x$代表高维稀疏的特征向量，包含user field，item field等
- 目标变量：$y$代表是否点击，$z$代表是否转化

预估目标：
- $pCVR=p(z=1|y=1, x)$
- 其中$pCTR=p(y=1|x)$，$pCTCVR=p(y=1,z=1|x)$
- 可以拆解为：$p(y=1,z=1|x)=p(y=1|x)\times p(z=1|y=1,x)$，即$pCTCVR=pCTR\times pCVR$

> $pCTCVR$是联合概率分布，$pCTR$是边缘概率分布，$pCVR$是条件概率分布

## 2.2 CVR Modeling and Challenges

传统CVR模型的样本是点击样本（它是整个曝光样本的子集），其中点击已转化为正样本，点击未转化为负样本。而后续CVR模型是用来预估整个曝光样本的，因此这样在建模和推断中就会出现bias。另外Data Sparsity也是一个问题，使得模型难以训练。

### Sample selection bias

传统CVR中，其通过引入辅助特征空间$\mathcal{X}_c$来进行近似建模$p(z=1|y=1,x)=p(z=1|x_c)$。其中$\mathcal{X}_c$是$\mathcal{S}_c$的有限集（$\mathcal{S}_c$是点击事件的样本空间）。

在训练阶段，$p(z=1|x_c)$是在点击样本$\mathcal{S}_c$中的$\mathcal{X}_c$空间中进行训练；但是在预测阶段，基于整个空间$\mathcal{X}$的$p(z=1|y=1,x)$计算的是$p(z=1|x)$，这样的等价是基于在pair对中$(x,y_x=1)$，对于任意$x\in \mathcal{X}$，$x\in \mathcal{X}_c$，即认为任意被曝光事件的特征都是属于被点击事件的特征空间的。但实际中这两个特征空间在很大概率是不满足这个假设的，因为$\mathcal{X}_c$只是$\mathcal{X}$的子集，在点击事件这个子集中提取特征是有偏的。

另外，在没有足够点击事件的情况下，$\mathcal{X}_c$与$\mathcal{X}$的特征空间是差别很大的，这样会带来与真实分布的不一致，导致CVR模型泛化能力较差。

### Data sparsity

曝光的商品数肯定是远大于被用户点击的商品数，通常大1~3个数量级。CTR使用的是曝光数据集，CVR使用的是点击数据集。正负样本极度不均衡。在淘宝的数据集上，有点击的样本空间仅占整体样本空间的4%，这就是DS问题，高度稀疏的训练数据使得模型的学习变得相当困难。

### Delayed feedback

用户点击后的转化是延时的，比如用户的购买行为可能是在点击的多天以后发生。作者提到在淘宝系统中的delay是可以被稍微接受，另一点是结合之前的work可以handle it，所以本文不去重点将delayed feedback问题。

> Olivier Chapelle. 2014. Modeling delayed feedback in display advertising. In
Proceedings of the 20th ACM SIGKDD international conference on Knowledge
discovery and data mining. ACM, 1097–1105.


## 2.3 Entire Space Multi-Task Model

ESSM模型整体结构如下：

![](pictures/esmm-model.png)

模型部分主要分为两个BASE Model：
- 左边是CVR模型
- 右边是CTR模型
- 他们share一个Embedding Layer

### Modeling over entire space

CVR预估可以写成以下公式：

$$
p(z=1|y=1,z)=\frac{p(y=1,z=1|x)}{p(y=1|x)}
$$

其中$p(y=1,z=1|x)$和$p(y=1|x)$都是在整个曝光样本集上进行建模的。这样可以直接解决Sample selection bias问题。

我们可以看到pCVR其实是pCTCVR和pCTR的商，那为什么不直接分别训练一个pCTCVR和pCTR模型，然后再求商得到CVR呢？原因在于，实际中pCTR是非常小的一个值，直接除一方面会带来numerical instability问题，另一方面还可能导致pCVR结果大于1。

因此ESMM直接采用multiplication form，pCVR只是中间一个变量，multiplication form使得这三个估计值可以一起训练，并且让模型可以探索到sequential pattern。另外，这样也保证了pCVR在[0,1]范围内，不会出现精度超过1的情况（这里应该是加了sigmoid激活函数）。

损失函数如下：

$$
L(\theta_{cvr},\theta_{ctr})=\sum_{i=1}^N l(y_i, x_i;\theta_{ctr})+\sum_{i=1}^N l(y_i\&z_i, f(x_i;\theta_{ctr})\times f(x_i;\theta_{cvr}))
$$

### Feature representation transfer

在ESMM中，CVR网络的Embedding Layer和CTR的Embedding Layer是shared，这是采用了一种迁移学习的范式。训练CTR的样本比CVR样本更加丰富，因此这种机制可以使得CVR网络从未点击数据中学习信息，帮助缓解Data Sparsity问题。

# 3 实验

![](pictures/esmm-result.png)

- BASE是ESMM左边的CVR模型部分，只在点击样本空间中训练
- DIVISION是分别训练CTR和CTCVR，再求除法
- ESMM-NS代表no share，即不去share feature representaion层
- ESMM是大模型

# 4 结论

本文提出了一种新的CVR预估模型ESMM，它能够很好好的使用user的sequential action信息。在CTR和CTCVR两个辅助任务的帮助下，ESMM可以很好的处理SSB和DS问题，并且在实际数据上证明了ESMM的有效性。未来作者将尝试设计一个global optimization models来capture整个从req->impression->click->conversion的流程。